# Run ARIMA Model
Step 1. Read dataset file<br>
Step 2. Run ARIMA model<br>
Step 3. Save output into file<br>
Step 4. Evaluate results

# 1. Read dataset file

## Install StatsModel and PDM ARIMA

In [1]:
# Install statsmodels version 0.12.1
# By the default version in Colab is 0.10.2 which doesn't have smapi.tsa.arima.ARIMA
!pip install statsmodels==0.12.1

# To determine the best parameter for each sector we need to run Auto ARIMA.
# See \Code\Build Models\ARIMA\20 Companies\ARIMA 20 companies.ipynb
# Auto ARIMA is installed from pmdarima (!pip install pmdarima). Statsmodel doesn't have Auto ARIMA.
# But now that we know the best parameters for each sector, we don't need Auto ARIMA.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.5 MB 14.4 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


## Import libraries

In [2]:
# Import file and data libraries
import math
import pickle
import numpy as np
import pandas as pd 

# Import ARIMA libraries
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
#from pmdarima.arima import auto_arima

# Supress warning due to ARIMA's date index
# e.g. An unsupported index was provided and will be ignored when forecasting
import warnings
warnings.filterwarnings("ignore")

# Import Matthews Correlation Coefficient module (MCC)
from sklearn.metrics import matthews_corrcoef

## Read dataset files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Read X & Y datasets from Pickle file, along with the metadata
dataset_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Dataset/"

with open(dataset_path + "Dataset.pickle", "rb") as f:
    X = pickle.load(f)
    Y = pickle.load(f) # Contains Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days
    Metadata = pickle.load(f) # Contains the Company ID, ticker, sector and Dataset ID
f.close()

# 2. Run ARIMA

## Set best model for each sector

In [5]:
# The best model for each sector was determined in 
# "Compare 6 methods V2 ARIMA Parameter Search.ipynb" and "ARIMA 20 companies.ipynb"
# Result: [5,0,0] is the best model for IT,IN,HC,CD but for FI it's [20,1,0]
# So we populate the Best Model List with those values.
Sector_List = ['IT', 'IN', 'FI', 'HC', 'CD']
Best_Model_List = []
for i in range(5):
    if i == 2: 
        Best_Model_List.append([20,1,0])
    else: 
        Best_Model_List.append([5,0,0])
print(Sector_List)
print(Best_Model_List)

['IT', 'IN', 'FI', 'HC', 'CD']
[[5, 0, 0], [5, 0, 0], [20, 1, 0], [5, 0, 0], [5, 0, 0]]


## Create functions

In [6]:
# Create a function to calculate 5% band i.e. 
#     0 to 5% = U1 (including 0), 5 to 10% = U2, 10 to 15% = U3, 15 to 20% = U4, >= U5.
#     <0 to -5% = D1, -5 to -10% = D2, -10 to -15% = D3, -15 to -20 = D4, <-20% = D5.
def Calc_Band(Input_Pct):
    if Input_Pct > 20: 
        band = 5
        prefix = 'U'
    elif Input_Pct < -20: 
        band = 5
        prefix = "D"
    elif Input_Pct >= 0:
        band = math.floor(Input_Pct / 5) + 1 # Floor(1/5) is 0 so to make it Band 1 we need to add 1
        prefix = "U"
    else:
        band = math.floor(Input_Pct / 5) # Floor(-1/5) is -1 so no need to add 1
        prefix = "D"
    return prefix + str(abs(band))

In [7]:
# Function to return the sector_id given the sector_code
def Get_Sector_ID(Sector_Code):
    for sector_id, sector_code in enumerate(Sector_List):
        if sector_code == Sector_Code:
            return sector_id

## Forecast for 1000 datasets

In [8]:
# Forecast for 1000 datasets, using the best model for each sector
# X, Y and Metadata contain 4000 datasets.
# Forecast the price band using the first 1000 datasets 
# Run time: 10 mins

F_Band_List   = [] # List of forecast band
Y_Band_List   = [] # List of actual band
F_Status_List = [] # List of forecast status
Output_File   = "" # List of output rows to be saved to file

# Forecast the price band for the first 1000 datasets (the remaining 2989 datasets are for ML training)
for ds in range(1000):

    # Create an ARIMA model with the best parameters for the sector
    # Parameter [5,0,0] is the best model for IT,IN,HC,CD but for FI it's [20,1,0]
    Sector_ID = Get_Sector_ID(Metadata[ds][2]) # Get the sector ID
    Model = SARIMAX( endog = X[ds]["HL_Avg"], 
                     order = (Best_Model_List[Sector_ID]),
                     initialization = 'approximate_diffuse' )

    # Fit the data on the ARIMA model
    Model_Fit = Model.fit()
    # print(Model_Fit.summary())

    # Y variable consists of: Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days
    Forecast_From = int(Y[ds][4]) # Y[4] is Y_Gap (number of days from X Last Price). Tipically 64.
    Forecast_To   = Forecast_From + int(Y[ds][5]) # Y[5] is Y_Days (number of days in a week for Y). Tipically 5.
    
    # Forecast for 3 months ahead, take the 1 week average
    F = Model_Fit.forecast(steps=Forecast_To) # Forecast to the end of the week (tipically 69)
    F_avg = F[Forecast_From:Forecast_To].mean() # Average the 1 week prices (tipically 65 to 69)

    # Compare to the last price in the 5Y period, get the price gap %, calculate the price band
    X_Last = int(Y[ds][1]) # Y(1) is the last price in X 5 year range
    F_Gap_Pct = (F_avg - X_Last) / X_Last * 100
    F_Band = Calc_Band(F_Gap_Pct)
    F_Band_List.append(F_Band) # Save the forecasted band into a list for MCC calculation

    # Compare the forecast to the actual in Y variable
    Y_Band = Y[ds][3] #Y[3] is Y_Band i.e. D1, U1, D2, U2, etc.
    Y_Band_List.append(Y_Band) # Save the Y_Band into a list for MCC calculation
    if F_Band == Y_Band: F_status = 1
    else: F_status = 0
    F_Status_List.append(F_status)
    
    # Collect the output rows into 1 variable to be written to the output file
    Output_Row = str(ds) + "," + F_Band + "," + Y_Band + "," + str(F_status) + "\n"
    print(Output_Row)
    Output_File = Output_File + Output_Row   

0,U5,U1,0

1,U5,D3,0

2,U5,U1,0

3,U5,U5,1

4,U5,U5,1

5,U5,D3,0

6,U5,U5,1

7,D1,U4,0

8,U5,U2,0

9,U1,U1,1

10,U5,D2,0

11,U5,U3,0

12,U5,D1,0

13,U5,D2,0

14,U5,U5,1

15,D1,D5,0

16,U5,D5,0

17,U5,U3,0

18,D1,U3,0

19,D1,U1,0

20,U3,D5,0

21,U1,U1,1

22,U5,U5,1

23,U5,D1,0

24,U5,D4,0

25,U1,U3,0

26,D5,D3,0

27,U5,U5,1

28,U5,D5,0

29,U5,U5,1

30,U3,U1,0

31,U5,U3,0

32,U5,U5,1

33,D1,U4,0

34,U5,D5,0

35,U5,U5,1

36,D4,U3,0

37,U5,U5,1

38,U5,U1,0

39,D1,D3,0

40,U1,U2,0

41,U5,D2,0

42,D1,U4,0

43,U5,U4,0

44,U4,U5,0

45,U4,U5,0

46,U5,U5,1

47,U5,D4,0

48,U5,D1,0

49,D5,D5,1

50,U5,U5,1

51,U5,D4,0

52,U5,U5,1

53,U5,D1,0

54,U3,U5,0

55,U5,U5,1

56,D1,D4,0

57,U3,U5,0

58,U5,U3,0

59,D1,U4,0

60,U1,U5,0

61,U5,U5,1

62,U5,U5,1

63,U5,D5,0

64,U5,U1,0

65,D1,D2,0

66,U5,U5,1

67,U5,U5,1

68,D1,D1,1

69,D1,U4,0

70,U5,U5,1

71,D1,D5,0

72,U5,U4,0

73,U5,U4,0

74,U5,D3,0

75,U5,D1,0

76,D4,U2,0

77,D1,U5,0

78,U5,U5,1

79,U5,U1,0

80,U5,U5,1

81,U5,D3,0

82,U5,U5,1

83,D1,U4,0

84

# 3. Save output into file

In [9]:
#print(output_file) 
file_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Output/"
f = open(file_path + "ARIMA.csv", "w")
f.write(Output_File)
f.close()

# 4. Evaluate results

## Correct percentage

In [10]:
print(sum(F_Status_List)) #125

157


## Matthew Correlation Coefficient

In [11]:
matthews_corrcoef(Y_Band_List, F_Band_List) # 0.0075768490861390925

0.03566265733565979